## **UAOPiloto**
Detección de estudiantes que van a desertar en 2 períodos.

TABLAS BASE:  DATOS

TABLAS MYSQL:
1. estudiantes_limpio:  es la tabla ESTUDIANTE pero ahora si limpia
2. periodosRegulares:   es la tabla de periodos y ya viene con un orden que me permite identificar " los siguientes dos periodos"
3.  

# **1.  Conexión a la base de datos**
Nos conectamos a la base de datos MYSQL

In [40]:
import pandas as pd
from sqlalchemy import create_engine

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
# Crea una instancia del motor SQLAlchemy
motor = create_engine(cadena_conexion)

# **2.  crear DATOS**

Se crea DATOS a partir de DISTINCT EstudianteAsignatura(id_estudiante,periodo)

In [41]:
query = """
    SELECT distinct id_estudiante, periodo
    FROM estudianteAsignatura
    ORDER BY id_estudiante, periodo
"""
DATOS = pd.read_sql(query, con=motor)

# NOTA:  falta manejar el caso de cursos de verano...  cuándo marcar al estudainte como desertor?  
# ERIK ya tiene la pregunta 20231009. R/  indica ERIK solo considerar períodos regulares

### **2.1 adicionamos el primer periodo que hizo matricula**
De la tabla EstudiantesMatriculas adicionamos el primer periodo en que el estudiante hizo su priemra matricula
EstudiantesMatriculas(id_estudiante, MIN(periodo))

In [42]:
# adicionamos el primer periodo en que ese estudiante hizo una matricula
query = """
    SELECT  id_estudiante, min(periodo) periodo_inicial
    FROM EstudiantesMatriculas
    WHERE periodo like '%1' or periodo like '%3'
    GROUP BY id_estudiante
"""
temp = pd.read_sql(query, con=motor)
# hacemos join entre DATOS y PRIMER_PERIODO para adicionar el periodo_inicial
DATOS = DATOS.merge(temp, on=['id_estudiante'], how='left')

### **2.2 Adicionamos fecha_grado, fecha_nacimiento, genero, colegio, ciudad_colegio**
De la tabla Estudiantes_Limpia saco esta información por estudiante y la adicionamos a DATOS

In [43]:
# ahora adicionamos el periodo de grado Y TODAS LAS COLUMNAS REQUERIDAS DESDE ESTUDIANTES_LIMPIO 
# pues ya hice todo el trabajo
# de limpieza.

query = """
    SELECT  id_estudiante, fecha_grado , fecha_nacimiento, genero, colegio, ciudad_colegio
    FROM estudiantes_limpio
"""
temp = pd.read_sql(query, con=motor)

# hacemos join con DATOS para adicionar la fecha de grado
DATOS = DATOS.merge(temp, on=['id_estudiante'], how='left')

### **2.3 adicionamos el número del periodo (periodo_orden)**

en la tabla PERIODOS está el orden de cada periodo.  vamos a adicionarlos.


In [62]:
query = """
    SELECT  pr.periodo, pr.orden periodo_orden, pr.fecha_inicio periodo_fi, pr.fecha_final periodo_ff
    from periodosRegulares pr
"""
temp = pd.read_sql(query, con=motor)

DATOS = DATOS.merge(temp, on=['periodo'], how='left')


SALVEMOS DATOS para que no nos toque procesar lo anterior

In [65]:
DATOS.to_excel('DATOS.xlsx')

### **2.3 Ahora si calculamos si dada una fila en d de DATOS, los siguientes 2 periodos no matriculó (ni se graduó)**

Ahora procedemos a generar LABEL: 1- el estudiante desertó.  0- el estudiante no desertó

hacemos un FOR en DATOS
por cada d (en el periodo i)
   miramos si i e i+1 tiene matricula: 
   si si:   lo marcamos como 0 -  
   si no:  miramos si se graduó en i, i+1 o i+2.  
           si si:  marcamos como 0
           si no:  marcamos como 1
   

In [4]:
import pandas as pd
from sqlalchemy import create_engine

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
# Crea una instancia del motor SQLAlchemy
motor = create_engine(cadena_conexion)
DATOS = pd.read_excel('DATOS.xlsx')

In [5]:
# el máximo periodo_orden es 12

from datetime import datetime, timedelta

DATOS['LABEL'] = -1   # asumimos que no se puede calcular

for indice, d in DATOS.iterrows():
    
    id_estudiante = d['id_estudiante']
    periodo_orden = d['periodo_orden']

    # print(id_estudiante, periodo_orden,'fecha_grado:', d['fecha_grado'], 'periodo_fi:', d['periodo_fi'], d['periodo_fi'] + timedelta(days=365))


    if periodo_orden >= 10:  # si es 11 o 12 no podemos aún calcular si si o si no
        d['LABEL'] = -1
        # print('     no se sabe')
    else:
        existe_periodo_1 =  DATOS[(DATOS['id_estudiante'] == id_estudiante) & (DATOS['periodo_orden'] == periodo_orden +1)]
        existe_periodo_2 =  DATOS[(DATOS['id_estudiante'] == id_estudiante) & (DATOS['periodo_orden'] == periodo_orden +2)]

        # si matriculo periodo 1 o periodo 2
        if len(existe_periodo_1)>0 or len(existe_periodo_2)>0:
            
            # como matriculó al menos uno de los dos siguientes periodos, no es desertor.
            DATOS.at[indice, 'LABEL'] = 0  

        else: 

            # si entra aquí es porque no matriculo ninguno de los dos siguientes períodos

            # si no tiene fecha de grado, es desertor.
            if pd.isna(d['fecha_grado']):
                DATOS.at[indice, 'LABEL'] = 1
            else:  
                
                # si sí tiene fecha de grado, pero es posterior a 1 año, es desertor.
                if d['fecha_grado'] > d['periodo_fi'] + timedelta(days=365):  
                    DATOS.at[indice, 'LABEL'] = 1
                else:
                    # si la fecha de grado ocurre antes de un año, NO es desertor.
                    DATOS.at[indice, 'LABEL'] = 0


In [7]:
DATOS.columns

Index(['Unnamed: 0', 'id_estudiante', 'periodo', 'periodo_inicial',
       'fecha_grado', 'fecha_nacimiento', 'genero', 'colegio',
       'ciudad_colegio', 'periodo_orden', 'periodo_fi', 'periodo_ff', 'LABEL'],
      dtype='object')

# **3.  Adicionar variables de interés**

El excel HIPOTESIS vs VARIABLES va explicando qué variables se van a ir adicionando

### **3.1 Resultados por áreas icfes**

In [8]:
query = """
    SELECT  id_estudiante, biologia, matematica, filosofia, fisica, historia, quimica, lenguaje, 
            geografia, idioma, interdisciplinar
    FROM icfes;
"""
temp = pd.read_sql(query, con=motor)

In [10]:
temp.biologia.value_counts()

biologia
1.00     10
58.00     8
51.00     7
53.00     7
56.00     6
52.00     6
66.00     6
60.00     6
45.00     5
55.00     5
54.00     5
48.00     4
57.00     4
47.00     4
63.00     4
50.00     4
71.00     3
39.00     3
69.00     3
65.00     3
72.00     3
62.00     3
68.00     3
49.00     3
70.00     3
43.00     2
40.00     2
76.00     2
36.00     2
59.00     2
67.00     2
44.00     2
64.00     2
31.00     1
34.00     1
37.00     1
47.03     1
42.00     1
41.00     1
61.00     1
38.00     1
46.00     1
75.00     1
Name: count, dtype: int64